In [9]:
import os
import pandas as pd
import requests
import os
from googlesearch import search
import time
from tqdm import tqdm

In [10]:
base_dir = os.getcwd()
output_dir = os.path.join(base_dir, "..", "..", "output")

file_path1 = os.path.join(output_dir, "data_for_searching_news - Copy.xlsx")
df1 = pd.read_excel(file_path1)

In [11]:
len(df1)

37

In [12]:
# Function to get news URLs using Google Search
def get_news_urls(query, date):
    formatted_date = pd.to_datetime(date).strftime("%Y-%m-%d")
    search_query = f"{query} after:{formatted_date} before:{formatted_date} site:news"

    urls = []
    try:
        for url in search(search_query, num_results=2, lang="en"):  # Fetch top 2 news URLs
            urls.append(url)
            time.sleep(10)  # Prevent blocking
    except Exception as e:
        print(f"Error fetching news for {query}: {e}")

    return urls

In [13]:
batch_size = 76
num_batches = len(df1) // batch_size + (1 if len(df1) % batch_size != 0 else 0)

In [14]:
for i in range(num_batches):
    print(f"Processing batch {i + 1}/{num_batches}...")

    # Select batch
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(df1))
    df_batch = df1.iloc[start_idx:end_idx].copy()

    # Apply function to batch
    tqdm.pandas()
    df_batch["news_urls"] = df_batch.progress_apply(lambda row: get_news_urls(row["Query_Text"], row["Date"]), axis=1)

    # Save each batch separately
    file_path = os.path.join(output_dir, f"news_batch_{i + 1}.csv")
    df_batch.to_csv(file_path, index=False)

    print(f"Saved batch {i + 1} to {file_path}")

    # Wait for 1 minute before processing the next batch (except after last batch)
    if i < num_batches - 1:
        print("Waiting for 1 minute to avoid rate limiting...")
        time.sleep(30)

print("✅ All batches processed and saved!")

Processing batch 1/1...


100%|██████████| 37/37 [13:14<00:00, 21.47s/it]

Saved batch 1 to C:\Users\Hp\Desktop\Data Science\My Projects\FakeNewsDetection\src\advanced\..\..\output\news_batch_1.csv
✅ All batches processed and saved!
